# Preprocess data from OFAS

## Set up the environment

In [1]:
import json
import logging
import pandas as pd
from typing import Dict, List, Any
from haystack.dataclasses import ByteStream, Document
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
import requests
from tqdm import tqdm
import re
import tiktoken
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import ast
from pydantic import BaseModel
from openai import AsyncOpenAI
from dataclasses import dataclass
import deepl
import asyncio

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", None)

In [3]:
llm_client = AsyncOpenAI()

In [4]:
MAX_CONTEXT_TOKENS = 120000
tokenizer = tiktoken.get_encoding("cl100k_base")

In [5]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

## Load the data

### From file

In [161]:
# Load the PDF paths from the JSON file
with open('data/sources/pdf_paths.json', 'r') as file:
    pdf_paths = json.load(file)
# pdf_paths = pdf_paths
pdf_paths = [pdf_paths[0]]

### From link

In [6]:
# Load the PDF URLs from the JSON file
with open('sources/pdf_urls.json', 'r') as file:
    pdf_urls = json.load(file)
pdf_urls = pdf_urls
# pdf_urls = pdf_urls[:20] + pdf_urls[460:480] + pdf_urls[-20:]
# pdf_urls = pdf_urls[:20] + pdf_urls[220:240]+ pdf_urls[430:450] + pdf_urls[460:480] + pdf_urls[930:950] + pdf_urls[1390:1410] + pdf_urls[-20:]

## Pre-processing

### LLM augmentation

#### HYQ generation

In [7]:
class HYQReformulationSchema(BaseModel):
    hyq: List[str]
    hyq_declarative: List[str]

In [8]:
QUERY_STATEMENT_REWRITING_PROMPT_DE = """<anweisungen>
    <anweisung>Gebe den untenstehenden <text> vor, formuliere {n_alt_queries} Fragen, die der Text genau beantworten kann</anweisung>
    <anweisung>Geben Sie die generierten Fragen vor, formulieren Sie sie in einem deklarativen/affirmativen Tonfall in mehrere alternative Aussagen um</anweisung>
    <anweisung>Jede umformulierte Aussage sollte die Bedeutung der ursprünglichen Anfrage beibehalten, sie aber auf eine etwas andere Weise ausdrücken</anweisung>
    <anweisung>Schreiben Sie Fragen/Reformulierungen immer in derselben Sprache wie der <text></anweisung>
</anweisungen>

<Beispiele>
hyq: [„Wie ist das Wetter?“, „Was ändert sich mit AHV21?“, „Was bedeutet das flexible Rentenalter?“]
hyq_delarative: [„Ich möchte wissen, wie das Wetter ist“, „Erklär mir, was sich mit der AHV21 ändert“, „Flexibles Rentenalter erklärt“]
</Beispiele>

<format_der_antwort>
HYQReformulationSchema(BaseModel)
    hyq: List[str] # eine Liste von Fragen, die der <text> genau beantworten kann.
    hyq_declarative = List[str] # die affirmative/deklarative Umformulierung der hyq-Fragen.
</format_der_antwort>

<text>
{text}
</text>"""

QUERY_STATEMENT_REWRITING_PROMPT_FR = """<instructions>
    <instruction>Étant donné le <texte> ci-dessous, formulez {n_alt_queries} questions auxquelles le texte peut exactement répondre</instruction>
    <instruction>Étant donné les questions générées, reformulez les en plusieurs énoncés alternatifs sur un ton déclaratif/affirmatif</instruction>
    <instruction>Chaque déclaration reformulée doit conserver le sens de la requête originale mais l'exprimer d'une manière légèrement différente</instruction>
    <instruction>Toujours écrire les questions/reformulations dans la même langue que le <texte></instruction>
</instructions>

<exemples>
hyq: ["Quel temps fait-il?", "Que change avec AVS21 ?", "Que signifie l'âge de la retraite flexible ?"]
hyq_delarative: ["J'aimerais connaître le temps qu'il fait", "Explique moi ce qui change avec AVS21", "L'âge de la retraite flexible expliqué"]
</exemples>

<format_de_réponse>
HYQReformulationSchema(BaseModel)
    hyq: List[str] # une liste de questions auxquelles le <texte> peut répondre exactement
    hyq_declarative = List[str] # la reformulation de manière affirmative/déclarative des questions hyq
</format_de_réponse>

<texte>
{text}
</texte>"""

QUERY_STATEMENT_REWRITING_PROMPT_IT = """<istruzioni>
    <istruzione>Dato il <testo> sottostante, formulare {n_alt_queries} domande a cui il testo può rispondere esattamente</istruzione>.
    <istruzione>Date le domande generate, riformularle in diverse affermazioni alternative con un tono dichiarativo/affermativo</istruzione>.
    <istruzione>Ogni affermazione riformulata deve mantenere il significato della domanda originale, ma esprimerlo in modo leggermente diverso</istruzione>.
    <istruzione>Scrivere sempre le domande/riformulazioni nella stessa lingua del <testo></istruzione>.
</istruzioni>

<esempi>
hyq: [“Com'è il tempo?”, “Cosa sta cambiando con AVS21?”, “Cosa significa l'età pensionabile flessibile?”]
hyq_delarative: [“Vorrei sapere com'è il tempo”, “Spiegami cosa sta cambiando con AVS21”, “L'età pensionabile flessibile spiegata”].
</esempi>

<formato_di_risposta>
HYQReformulationSchema(BaseModel)
    hyq: List[str] # un elenco di domande a cui il <testo> può rispondere esattamente
    hyq_declarative = List[str] # la riformulazione affermativa/declarativa delle domande hyq.
</formato_di_risposta>

<testo>
{text}
</testo>"""

hyq_prompts = {
    "de": QUERY_STATEMENT_REWRITING_PROMPT_DE,
    "fr": QUERY_STATEMENT_REWRITING_PROMPT_FR,
    "it": QUERY_STATEMENT_REWRITING_PROMPT_IT,
}

#### Summary generation

In [9]:
class SummarySchema(BaseModel):
    summary: str

In [10]:
QUERY_STATEMENT_SUMMARY_PROMPT_DE = """<anweisungen>
    <anweisung>Gebe den untenstehenden <text> vor, und formuliere eine Zusammenfassung, die den Inhalt des Textes präzise wiedergibt</anweisung>
    <anweisung>Die Zusammenfassung sollte in einem klaren und prägnanten Tonfall erfolgen</anweisung>
    <anweisung>Die Zusammenfassung muss den wesentlichen Inhalt des Textes erfassen und ihn in eigenen Worten wiedergeben</anweisung>
    <anweisung>Schreiben Sie die Zusammenfassung immer in derselben Sprache wie der <text></anweisung>
</anweisungen>

<Beispiele>
summary: "Das Individuelle Konto (IK) ist entscheidend für die Rentenberechnung in der AHV. Es erfasst alle Einkommen, Beitragszeiten und Betreuungsgutschriften, die für Alters-, Hinterlassenen- oder Invalidenrenten relevant sind. Fehlende Beitragsjahre, auch als Beitragslücken bezeichnet, können zu einer Kürzung der Versicherungsleistungen führen. Die Ausgleichskassen führen für jede versicherte Person ein IK und sind verantwortlich für die korrekte Erfassung der Beiträge."
</Beispiele>

<format_der_antwort>
SummarySchema(BaseModel)
    summary: str # Eine Zusammenfassung, die den <text> exakt wiedergibt
</format_der_antwort>

<text>
{text}
</text>"""

QUERY_STATEMENT_SUMMARY_PROMPT_FR = """<instructions>
    <instruction>Étant donné le <texte> ci-dessous, formulez un résumé qui restitue précisément le contenu du texte</instruction>
    <instruction>Le résumé doit être présenté de manière claire et concise</instruction>
    <instruction>Le résumé doit capturer l'essence du contenu du texte en le reformulant avec vos propres mots</instruction>
    <instruction>Rédigez le résumé toujours dans la même langue que le <texte></instruction>
</instructions>

<exemples>
summary: "Le Compte Individuel (CI) est déterminant pour le calcul de la retraite dans l’AVS. Il recense tous les revenus, périodes de cotisation et crédits de prise en charge qui sont pertinents pour les rentes de vieillesse, de survivants ou d'invalidité. Des années de cotisation manquantes, également appelées lacunes de cotisation, peuvent entraîner une réduction des prestations. Les caisses de compensation tiennent un CI pour chaque assuré et sont responsables de l’enregistrement correct des contributions."
</exemples>

<format_de_réponse>
SummarySchema(BaseModel)
    summary: str # Un résumé qui restitue précisément le contenu du <texte>
</format_de_réponse>

<texte>
{text}
</texte>"""

QUERY_STATEMENT_SUMMARY_PROMPT_IT = """<istruzioni>
    <istruzione>Dato il <testo> sottostante, formula un riassunto che riporti esattamente il contenuto del testo</istruzione>
    <istruzione>Il riassunto deve essere presentato in modo chiaro e conciso</istruzione>
    <istruzione>Il riassunto deve catturare l'essenza del contenuto del testo riformulandolo con parole proprie</istruzione>
    <istruzione>Scrivi il riassunto sempre nella stessa lingua del <testo></istruzione>
</istruzioni>

<esempi>
summary: "Il Conto Individuale (CI) è fondamentale per il calcolo delle pensioni nell'AVS. Esso registra tutti i redditi, periodi di contribuzione e crediti per l'assistenza che sono rilevanti per le pensioni di vecchiaia, superstiti o invalidità. Anni di contribuzione mancanti, noti anche come lacune contributive, possono portare a una riduzione delle prestazioni. Le casse di compensazione mantengono un CI per ogni assicurato e sono responsabili della corretta registrazione dei contributi."
</esempi>

<formato_di_risposta>
SummarySchema(BaseModel)
    summary: str # Un riassunto che riporti esattamente il contenuto del <testo>
</formato_di_risposta>

<testo>
{text}
</testo>"""

summary_prompts = {
    "de": QUERY_STATEMENT_SUMMARY_PROMPT_DE,
    "fr": QUERY_STATEMENT_SUMMARY_PROMPT_FR,
    "it": QUERY_STATEMENT_SUMMARY_PROMPT_IT,
}

### Parsing

#### From file

In [155]:
import os

# Define the parser class
class OFASParser:
    def __init__(self):
        self.pdf_converter = PyPDFToDocument()
        self.cleaner = DocumentCleaner(
            remove_empty_lines=True,
            remove_extra_whitespaces=True,
            remove_repeated_substrings=False,
        )
        self.splitter = DocumentSplitter(
            split_by="sentence",
            split_length=5,
            split_overlap=1,
            split_threshold=4,
        )

    def clean_text(self, text: str, link: str) -> str:
        # Remove headers (allowing newlines and spaces)
        fedlex_header_pattern = r'Accueil Recueil systématique.*?\)\s*\ue910\s*\ue910\s*\ue910\s*\ue910'
        text = re.sub(fedlex_header_pattern, '', text, flags=re.DOTALL)
        print(repr(text))

        # Remove Fedlex footer (date + Fedlex + link + pages)
        footer_pattern = (
            r'\d{2}/\d{2}/\d{4} \d{2}:\d{2} .*?\| Fedlex\s*'
            r'https?://\S+ \d+/\d+'
        )
        text = re.sub(footer_pattern, '', text, flags=re.DOTALL)
        
        # Remove isolated superscript-like lines (end of page)
        text = re.sub(r'^\s*(\d+\s*er|\d+|er)\s*$', '', text, flags=re.MULTILINE)
        text = re.sub(r'\.\s*\d+(?:\s*\d+)*\s+(?=[A-Z])', '. ', text, flags=re.MULTILINE)

        

            
        # Remove excess dots and formatting artifacts
        text = re.sub(r'\.{4,}', '', text)  # Remove sequences of four or more dots
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single newline
        
        text = text.strip()  # Remove leading and trailing whitespace
        return text

    async def generate_hyq(self, language: str, text: str) -> list:
        prompt = hyq_prompts.get(language)
        messages = [
            {"role": "user", "content": prompt.format(n_alt_queries=3, text=text)}
        ]
        res = await llm_client.beta.chat.completions.parse(
            model="gpt-4o",
            temperature=0,
            top_p=0.95,
            max_tokens=2048,
            messages=messages,
            response_format=HYQReformulationSchema,
        )
        
        hyq = res.choices[0].message.parsed.hyq
        hyq_declarative = res.choices[0].message.parsed.hyq_declarative
        return [hyq, hyq_declarative]

    async def generate_summary(self, language: str, text: str) -> list:
        prompt = summary_prompts.get(language)
        messages = [
            {"role": "user", "content": prompt.format(text=text)}
        ]
        res = await llm_client.beta.chat.completions.parse(
            model="gpt-4o",
            temperature=0,
            top_p=0.95,
            max_tokens=2048,
            messages=messages,
            response_format=SummarySchema,
        )
        
        summary = res.choices[0].message.parsed.summary
        return summary

    async def convert_to_documents(self, pdf_files: List[dict]) -> List[dict]:
        documents = []
        for file_info in tqdm(pdf_files, desc="Processing PDFs", unit="file"):
            try:
                file_path = "data/sources/fedlex/" + file_info['path']
                # print(file_path)
                # Check if file exists
                if not os.path.exists(file_path):
                    logger.warning(f"File not found: {file_path}")
                    continue

                # Check if file is a PDF
                if not file_path.lower().endswith('.pdf'):
                    logger.warning(f"Skipping non-PDF file: {file_path}")
                    continue

                # Read the PDF content
                with open(file_path, 'rb') as f:
                    pdf_content = f.read()

                # Convert PDF content to Document objects
                byte_stream = ByteStream(data=pdf_content)
                result = self.pdf_converter.run(sources=[byte_stream])
                converted_docs = result["documents"]

                # Clean documents
                cleaned_result = self.cleaner.run(documents=converted_docs)
                cleaned_docs = cleaned_result["documents"]

                # Split documents
                split_result = self.splitter.run(documents=cleaned_docs)
                split_docs = split_result["documents"]

                # For each cleaned document
                for doc in cleaned_docs:
                    cleaned_text = self.clean_text(doc.content, file_info['url'])

                    # Tokenize
                    tokenized_text = tokenizer.encode(cleaned_text)
                    if len(tokenized_text) > MAX_CONTEXT_TOKENS:
                        chunks = [tokenizer.decode(tokenized_text[i:i + MAX_CONTEXT_TOKENS]) for i in range(0, len(tokenized_text), MAX_CONTEXT_TOKENS)]
                        for chunk in chunks:
                            hyq = await self.generate_hyq('fr', chunk)
                            summary = await self.generate_summary('fr', chunk)
                            document = {
                                "url": file_info['url'],
                                "text": chunk,
                                "language": 'fr',
                                "tags": file_info['tag'],
                                "subtopics": file_info['subtopics'],
                                # "summary": "",
                                "summary": summary,
                                "doctype": "context_doc",
                                "organizations": "OFAS",
                                # "hyq": "",
                                "hyq": "{SEP}".join(hyq[0]),
                                # "hyq_declarative": ""
                                "hyq_declarative": "{SEP}".join(hyq[1])
                            }
                            documents.append(document)
                    else:
                        hyq = await self.generate_hyq('fr', cleaned_text)
                        summary = await self.generate_summary('fr', cleaned_text)
                        document = {
                            "url": file_info['url'],
                            "text": cleaned_text,
                            "language": 'fr',
                            "tags": file_info['tag'],
                            "subtopics": file_info['subtopics'],
                            # "summary": "",
                            "summary": summary,
                            "doctype": "context_doc",
                            "organizations": "OFAS",
                            # "hyq": "",
                            "hyq": "{SEP}".join(hyq[0]),
                            # "hyq_declarative": ""
                            "hyq_declarative": "{SEP}".join(hyq[1])
                        }
                        documents.append(document)
            except Exception as e:
                logger.error(f"Failed to process PDF {file_info}: {e}")
        return documents


#### From link

In [11]:
# Define the parser class
class OFASParser:
    def __init__(self):
        self.pdf_converter = PyPDFToDocument()
        self.cleaner = DocumentCleaner(
            remove_empty_lines=True,
            remove_extra_whitespaces=True,
            remove_repeated_substrings=False,
        )
        self.splitter = DocumentSplitter(
            split_by="sentence",
            split_length=5,
            split_overlap=1,
            split_threshold=4,
        )

    def clean_text(self, text: str) -> str:
        # Remove excess dots and formatting artifacts
        text = re.sub(r'\.{4,}', '', text)  # Remove sequences of three or more dots
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single newline
        text = text.strip()  # Remove leading and trailing whitespace
        return text

    async def generate_hyq(self, language: str, text: str) -> list:
        prompt = hyq_prompts.get(language)
        messages = [
            {"role": "user", "content": prompt.format(n_alt_queries=3, text=text)}
        ]
        res = await llm_client.beta.chat.completions.parse(
            model="gpt-4o",
            temperature=0,
            top_p=0.95,
            max_tokens=2048,
            messages=messages,
            response_format=HYQReformulationSchema,
        )
        
        hyq = res.choices[0].message.parsed.hyq
        hyq_declarative = res.choices[0].message.parsed.hyq_declarative
        return [hyq, hyq_declarative]

    async def generate_summary(self, language: str, text: str) -> list:
        prompt = summary_prompts.get(language)
        messages = [
            {"role": "user", "content": prompt.format(text=text)}
        ]
        res = await llm_client.beta.chat.completions.parse(
            model="gpt-4o",
            temperature=0,
            top_p=0.95,
            max_tokens=2048,
            messages=messages,
            response_format=SummarySchema,
        )
        
        summary = res.choices[0].message.parsed.summary
        return summary

    async def convert_to_documents(self, pdf_urls: List[dict]) -> List[dict]:
        documents = []
        for url in tqdm(pdf_urls, desc="Processing PDFs", unit="file"):
            # print(url)
            try:
                # Fetch the PDF content
                response = requests.get(url['url'])
                response.raise_for_status()

                # Check if the content is a PDF
                if response.headers.get('Content-Type') != 'application/pdf':
                    logger.warning(f"Skipping non-PDF content from {url['url']}")
                    continue

                pdf_content = response.content

                # Convert PDF content to Document objects
                byte_stream = ByteStream(data=pdf_content)
                result = self.pdf_converter.run(sources=[byte_stream])
                converted_docs = result["documents"]  # Ensure this is a list of Document objects

                # Clean documents
                cleaned_result = self.cleaner.run(documents=converted_docs)
                cleaned_docs = cleaned_result["documents"]  # Extract the list of cleaned Document objects

                # Split documents
                split_result = self.splitter.run(documents=cleaned_docs)
                split_docs = split_result["documents"]  # Extract the list of split Document objects

                # Create document format for each converted document
                for doc in cleaned_docs:
                    # Clean the text content
                    cleaned_text = self.clean_text(doc.content)

                    # Check if the cleaned text exceeds the token limit
                    tokenized_text = tokenizer.encode(cleaned_text)
                    if len(tokenized_text) > MAX_CONTEXT_TOKENS:
                        chunks = [tokenizer.decode(tokenized_text[i:i + MAX_CONTEXT_TOKENS]) for i in range(0, len(tokenized_text), MAX_CONTEXT_TOKENS)]
                        for chunk in chunks:
                            hyq = await self.generate_hyq(language, chunk)
                            summary = await self.generate_summary(language, chunk)
                            document = {
                                "url": url['url'],
                                "text": chunk,
                                "language": language,
                                "tags": url['tag'],
                                "subtopics": url['subtopics'],
                                "summary": summary,  # TODO
                                "doctype": "context_doc",  # Constant value
                                "organizations": "OFAS",  # Constant value
                                "hyq": "{SEP}".join(hyq[0]),
                                "hyq_declarative": "{SEP}".join(hyq[1])
                            }
                            documents.append(document)
                    else:
                        # Extract language from the URL
                        language = url['url'].split('/')[3]
                        hyq = await self.generate_hyq(language, cleaned_text)
                        summary = await self.generate_summary(language, cleaned_text)
                        document = {
                            "url": url['url'],
                            "text": cleaned_text,
                            "language": language,
                            "tags": url['tag'],
                            "subtopics": url['subtopics'],
                            "summary": summary,  # TODO
                            "doctype": "context_doc",  # Constant value
                            "organizations": "OFAS",  # Constant value
                            "hyq": "{SEP}".join(hyq[0]),
                            "hyq_declarative": "{SEP}".join(hyq[1])
                        }
                        documents.append(document)
            except requests.RequestException as e:
                logger.error(f"Failed to fetch PDF from {url}: {e}")
        return documents

#### With layout handling

In [162]:
import pdfplumber
import re
from collections import defaultdict

class OFASParser:
    def __init__(self):
        self.pdf_converter = PyPDFToDocument()
        self.cleaner = DocumentCleaner(
            remove_empty_lines=True,
            remove_extra_whitespaces=True,
            remove_repeated_substrings=False,
        )
        self.splitter = DocumentSplitter(
            split_by="sentence",
            split_length=5,
            split_overlap=1,
            split_threshold=4,
        )

    def parse_pdf_with_layout(self, file_path: str) -> str:
        text_blocks = []
        try:
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    # Group chars into lines based on y-coordinate
                    lines = defaultdict(list)
                    for char in page.chars:
                        line_y = round(char['top'], 1)
                        lines[line_y].append(char)
                    
                    # Sort lines by vertical position
                    for y in sorted(lines.keys()):
                        line_chars = sorted(lines[y], key=lambda c: c['x0'])
                        line_text = ''.join([c['text'] for c in line_chars])
                        
                        # Detect superscript numbers (small font)
                        small_numbers = [c for c in line_chars if c['size'] < 8 and c['text'].isdigit()]
                        if small_numbers:
                            # Reinsert small numbers at start of line (or adjust as needed)
                            numbers = ''.join(c['text'] for c in small_numbers)
                            line_text = numbers + ' ' + line_text
                        
                        text_blocks.append(line_text)
            full_text = '\n'.join(text_blocks)
            return full_text
        except Exception as e:
            logger.error(f"Failed to parse PDF with layout: {file_path}: {e}")
            return ""

    def clean_text(self, text: str, link: str) -> str:
        # Your existing cleaning logic remains
        fedlex_header_pattern = r'Accueil Recueil systématique.*?\)\s*\ue910\s*\ue910\s*\ue910\s*\ue910'
        text = re.sub(fedlex_header_pattern, '', text, flags=re.DOTALL)

        footer_pattern = (
            r'\d{2}/\d{2}/\d{4} \d{2}:\d{2} .*?\| Fedlex\s*'
            r'https?://\S+ \d+/\d+'
        )
        text = re.sub(footer_pattern, '', text, flags=re.DOTALL)

        # Remove isolated superscript-like lines (after correct merging)
        text = re.sub(r'^\s*(\d+\s*er|\d+|er)\s*$', '', text, flags=re.MULTILINE)
        text = re.sub(r'\.\s*\d+(?:\s*\d+)*\s+(?=[A-Z])', '. ', text, flags=re.MULTILINE)

        # Remove formatting artifacts
        text = re.sub(r'\.{4,}', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', '\n', text)
        text = text.strip()
        return text

    async def convert_to_documents(self, pdf_files: List[dict]) -> List[dict]:
        documents = []
        for file_info in tqdm(pdf_files, desc="Processing PDFs", unit="file"):
            try:
                file_path = "data/sources/fedlex/" + file_info['path']
                if not os.path.exists(file_path):
                    logger.warning(f"File not found: {file_path}")
                    continue

                # Extract text preserving layout
                raw_text = self.parse_pdf_with_layout(file_path)
                if not raw_text:
                    continue

                cleaned_text = self.clean_text(raw_text, file_info['url'])

                # Tokenize
                tokenized_text = tokenizer.encode(cleaned_text)
                if len(tokenized_text) > MAX_CONTEXT_TOKENS:
                    chunks = [tokenizer.decode(tokenized_text[i:i + MAX_CONTEXT_TOKENS]) for i in range(0, len(tokenized_text), MAX_CONTEXT_TOKENS)]
                    for chunk in chunks:
                        # hyq = await self.generate_hyq('fr', chunk)
                        # summary = await self.generate_summary('fr', chunk)
                        document = {
                            "url": file_info['url'],
                            "text": chunk,
                            "language": 'fr',
                            "tags": file_info['tag'],
                            "subtopics": file_info['subtopics'],
                            "summary": "",
                            # "summary": summary,
                            "doctype": "context_doc",
                            "organizations": "OFAS",
                            "hyq": "",
                            # "hyq": "{SEP}".join(hyq[0]),
                            "hyq_declarative": ""
                            # "hyq_declarative": "{SEP}".join(hyq[1])
                        }
                        documents.append(document)
                else:
                    # hyq = await self.generate_hyq('fr', cleaned_text)
                    # summary = await self.generate_summary('fr', cleaned_text)
                    document = {
                        "url": file_info['url'],
                        "text": cleaned_text,
                        "language": 'fr',
                        "tags": file_info['tag'],
                        "subtopics": file_info['subtopics'],
                            "summary": "",
                            # "summary": summary,
                            "doctype": "context_doc",
                            "organizations": "OFAS",
                            "hyq": "",
                            # "hyq": "{SEP}".join(hyq[0]),
                            "hyq_declarative": ""
                            # "hyq_declarative": "{SEP}".join(hyq[1])
                    }
                    documents.append(document)
            except Exception as e:
                logger.error(f"Failed to process PDF {file_info}: {e}")
        return documents


ModuleNotFoundError: No module named 'pdfplumber'

#### Initialize the parsing

In [156]:
# Initialize the parser
ofas_parser = OFASParser()

##### from file

In [157]:
# Process the PDF URLs
async def process_pdfs():
    # Convert PDFs to documents
    documents = await ofas_parser.convert_to_documents(pdf_paths)
    # documents = await ofas_parser.convert_to_documents(pdf_urls)
    
    # Output the documents
    # for doc in documents[:2]:
    #     print(doc)
    
    return documents

##### From link

In [32]:
# Process the PDF URLs
async def process_pdfs():
    # Convert PDFs to documents
    documents = await ofas_parser.convert_to_documents(pdf_urls)
    # documents = await ofas_parser.convert_to_documents(pdf_urls)
    
    # Output the documents
    # for doc in documents[:2]:
    #     print(doc)
    
    return documents


#### run the processing function

In [158]:
documents = await process_pdfs()

Processing PDFs:   0%|          | 0/13 [00:00<?, ?file/s]

'\n281.41\nOrdonnance\nconcernant la saisie et la réalisation de\nparts\xa0de\xa0communautés\n(OPC)\ndu 17 janvier 1923 (État le 1 janvier 2017) Nouvelle teneur selon le ch. I de l’O du 29 juin 2016, en vigueur depuis le 1\xa0janv.\xa02017 (RO 2016 2643).\nLe Conseil fédéral suisse,\nvu l’art. 15, al. 2, de la loi fédérale du 11 avril 1889 sur la poursuite pour dettes\xa0et\xa0la\xa0faillite\n(LP),\nordonne: RS 281.1 Nouvelle teneur selon le ch. I de l’O du 29 juin 2016, en vigueur depuis le 1\xa0janv.\xa02017 (RO 2016 2643).\nI. Saisie\nObjet de la saisie\nArt. 1 La saisie des droits du débiteur dans une succession non partagée, dans une indivision,\ndans une société en nom collectif, dans une société en commandite ou dans une\ncommunauté analogue, ne peut porter que sur le produit lui revenant dans la liquidation\nde la communauté, lors même que celle-ci ne s’étend qu’à une chose unique.1\ner\n1 er\n23\n2\n3 er\n117/03/2025 14:26 RS 281.41 - Ordonnance du 17 janvier 1923 concer... | 

2025-03-18 17:48:07,541 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-18 17:48:10,896 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing PDFs:   8%|▊         | 1/13 [00:07<01:28,  7.38s/file]

'\n830.1\nLoi fédérale\nsur la partie générale du droit des assurances sociales\n(LPGA)\ndu 6 octobre 2000 (État le 1 janvier 2024)\nL’Assemblée fédérale de la Confédération suisse,\nvu les art. 112, al. 1, 114, al. 1, et 117, al. 1, de la Constitution,\nvu le rapport d’une commission du Conseil des États du 27 septembre 1990,\nvu les avis du Conseil fédéral des 17 avril 1991, 17 août 1994 et 26 mai 1999,\nvu le rapport de la Commission de la sécurité sociale et de la santé publique\ndu\xa0Conseil\xa0national du\xa026\xa0mars\xa01999,\narrête: RS 101 FF 1991 II 181 FF 1991 II 888 FF 1994 V 897 Non publié dans la FF, cf. BO 1999 N 1241 et 1244 FF 1999 4168\nChapitre 1 Champ d’application\nArt. 1 But et objet\nLa présente loi coordonne le droit fédéral des assurances sociales:er\n1\n2\n3 4 5\n6\n1\n2\n3\n4\n5\n617/03/2025 14:24 RS 830.1 - Loi fédérale du 6 octobre 2000 sur la... | Fedlex\nhttps://www.fedlex.admin.ch/eli/cc/2002/510/fr?print=true 1/29\x0ca.\nb.\nc.\nd.en définissant les p

2025-03-18 17:48:16,648 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-18 17:48:23,513 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing PDFs:  15%|█▌        | 2/13 [00:19<01:55, 10.46s/file]

...

2025-03-18 17:50:49,141 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-18 17:50:55,874 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing PDFs:  85%|████████▍ | 11/13 [02:52<00:31, 15.60s/file]

'\n836.2\nLoi fédérale\nsur les allocations familiales et les aides financières\nallouées\xa0aux organisations familiales\n(Loi sur les allocations familiales, LAFam)\ndu 24 mars 2006 (État le 1 janvier 2025) Les termes désignant des personnes s’appliquent également aux femmes et aux hommes. Nouvelle teneur selon le ch.\xa0I de la LF du 27\xa0sept.\xa02019, en vigueur depuis le 1\xa0août\xa02020 (RO 2020 2775;\nFF 2019 997).\nL’Assemblée fédérale de la Confédération suisse,\nvu l’art.\xa0116, al.\xa01, 2 et 4, de la Constitution,\nvu le rapport de la Commission de la sécurité sociale et de la santé publique du\xa0Conseil\nnational du 20 novembre 1998 et le rapport complémentaire du\xa08\xa0septembre 2004,\nvu les avis du Conseil fédéral du 28 juin 2000 et du 10 novembre 2004,\narrête: RS 101 Nouvelle teneur selon le ch.\xa0I de la LF du 27\xa0sept.\xa02019, en vigueur depuis le 1\xa0août\xa02020 (RO 2020 2775;\nFF 2019 997). FF 1999 2942 FF 2004 6459 FF 2000 4422 FF 2004 6513\x001\n2\n

2025-03-18 17:51:00,012 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-18 17:51:04,256 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing PDFs:  92%|█████████▏| 12/13 [03:00<00:13, 13.40s/file]

'\n836.21\nOrdonnance\nsur les allocations familiales\n(OAFam)\ndu 31 octobre 2007 (État le 1 janvier 2025)\nLe Conseil fédéral suisse,\nvu les art.\xa04, al. 3, 13, al.\xa04, 21b, al.\xa01, 21e et 27, al.\xa01, de la loi du 24 mars 2006 sur les\nallocations familiales (LAFam),\narrête: RS 836.2 Nouvelle teneur selon le ch. I de l’O du 8 sept. 2010, en vigueur depuis le 15 oct. 2010 (RO 2010 4495).\nSection 1 Dispositions générales\nArt. 1 Allocation de formation\n(art. 3, al. 1, let. b, LAFam) Un droit à l’allocation de formation existe pour les enfants accomplissant une formation\nau sens des art. 49 et 49 du règlement du 31 octobre 1947 sur l’assurance-vieillesse et\nsurvivants. Est considérée comme formation postobligatoire la formation qui suit la scolarité\nobligatoire. La durée et la fin de la scolarité obligatoire sont régies par les dispositions de\nchaque canton. Nouvelle teneur selon le ch.\xa0I de l’O du 19\xa0juin\xa02020, en vigueur depuis le 1\xa0août\xa02020 (RO 2020 27

2025-03-18 17:51:08,980 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-18 17:51:13,330 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing PDFs: 100%|██████████| 13/13 [03:09<00:00, 14.60s/file]


In [153]:
documents

[{'url': 'https://www.fedlex.admin.ch/eli/cc/39/55_55_57/fr',
  'text': '281. Ordonnance concernant la saisie et la réalisation de parts de communautés (OPC) du 17 janvier 1923 (État le 1 janvier 2017) Nouvelle teneur selon le ch. I de l’O du 29 juin 2016, en vigueur depuis le 1 janv. 2017 (RO 2016 2643). Le Conseil fédéral suisse, vu l’art. 15, al. 2, de la loi fédérale du 11 avril 1889 sur la poursuite pour dettes et la faillite (LP), ordonne: RS 281. Nouvelle teneur selon le ch. I de l’O du 29 juin 2016, en vigueur depuis le 1 janv. 2017 (RO 2016 2643). I. Saisie Objet de la saisie Art. La saisie des droits du débiteur dans une succession non partagée, dans une indivision, dans une société en nom collectif, dans une société en commandite ou dans une communauté analogue, ne peut porter que sur le produit lui revenant dans la liquidation de la communauté, lors même que celle-ci ne s’étend qu’à une chose unique. Cette disposition s’applique également à la part que possède le débiteur d

### Save the document into a file

In [160]:
# Save documents to a CSV file
df = pd.DataFrame(documents)  # Create a DataFrame from the list of documents
df.to_csv('data/output/fedlex.csv', index=False)  # Save to CSV without the index